## Planning Graph

In [149]:
from geojson import LineString, GeometryCollection, Point
from math import radians, sin, cos, sqrt, asin
import heapq
import numpy as np
from osmread import parse_file, Way, Node

class GraphPlan(object):
    """ Process data for a country to produce data structures needed for
        a graph search
    """
    # TODO: Class excepts any bz2 file for processing 
    def __init__(self):
        """
        Connect to OSM data base and prepare basic data structures
        """
        #TODO: Allow user to specify database, user and national region
        conn_string = "host='localhost' dbname='osm' user='dcromp'"
        print ("Connecting to database\n->%s" % (conn_string))
        # get a connection, if a connect cannot be made an exception will be raised here
        conn = psycopg2.connect(conn_string)
        # conn.cursor will return a cursor object
        cursor = conn.cursor()
        print ("Connected!\n")
        
        # User guide on using osmosis tags
        # http://skipperkongen.dk/2012/08/02/examples-of-querying-a-osm-postgresql-table-with-the-hstore-tags-column/
        
    def node_neighbours(self, node_id):
        """
        For a give node_id its neighbouring nodes are returned 
        Args:
            Node_id
        Returns:
            [node_id] List of neighbouring node id's  
        """
        #Query to get way which the node occurs in
        query_string = "SELECT * FROM way_nodes WHERE node_id = {}".format(node_id)
        cursor.execute(query_string)
        records = cursor.fetchall()
        #Extract way id's from the records
        ways = [record[0] for record in records]
        neighbours_list = set()
        for way in ways:
            #Make sure we return a road using highway key and not a polygon of a building
            query_string = "SELECT nodes FROM ways WHERE id = {} AND exist(tags, 'highway')".format(way)
            cursor.execute(query_string)
            records = cursor.fetchall()
            try: #Possiable that way is not a road so nodes is empty
                nodes = records[0][0]
            except:
                continue #Skip way if way is not a road
            node_index = nodes.index(node_id)
            #Extract the nodes neighbour and save to dict
            forward_node = node_index + 1
            backward_node = node_index - 1
            try:
                forward = nodes[forward_node]
                neighbours_list.add(forward)
            except:
                pass
            if backward_node >= 0:
                try:
                    backward = nodes[backward_node]
                    neighbours_list.add(backward)
                except:
                    pass 
        return list(neighbours_list)
    
    def node_path_geojson(self, node_list):
        roads = []
        for node_id in node_list:
            query_string = "SELECT ST_AsText(ST_Transform(geom,4326)) FROM nodes WHERE id = {}".format(node_id)
            cursor.execute(query_string)
            records = cursor.fetchall()
            lng, lat = records[0][0].replace('POINT(', ' ').replace(')', '').split(' ')[1:3]
            roads.append((float(lng), float(lat)))
        return LineString(roads)
    
class PriorityQueue:
    def __init__(self):
        self.elements = []
    
    def empty(self):
        return len(self.elements) == 0
    
    def put(self, item, priority):
        heapq.heappush(self.elements, (priority, item))
    
    def get(self):
        return heapq.heappop(self.elements)[1]
    
class BusGraph(GraphPlan):
    """
    Turns bus routes into nodes for the graph search
    """
    def __init__(self, lon1,  lat1, lon2, lat2):
        GraphPlan.__init__(self, lon1,  lat1, lon2, lat2 )
        
        #Extract all raw bus routes
        self.route_data = [row for row in self.map_data if row['type'] == 'route']
        #self.route_data = [row for row in self.way_data if 'route' in row['data']['tag'].keys()]
        #self.bus_data = [row for row in self.route_data if 'bus' in row['data']['tag']['route']]
        
graph = GraphPlan()

Connecting to database
->host='localhost' dbname='osm' user='dcromp'
Connected!



## Search Algorithms

In [150]:
from time import time
from collections import deque

def breadth_first_search(graph, start, goal):
    frontier = deque([])
    frontier.append(start)
    came_from = {}
    came_from[start] = None
    
    while len(frontier) > 0:
        current = frontier.popleft()
        if current == goal:
            print("Solution Found")
            break
        children = graph.node_neighbours(current)
        for child in children:
            if child not in came_from:
                frontier.append(child)
                came_from[child] = current
    return came_from

def uniform_cost_search(graph, start, goal):
    frontier = PriorityQueue()
    frontier.put(start, 0)
    came_from = {}
    cost_so_far = {}
    
    came_from[start] = None
    cost_so_far[start] = 0
    
    while not frontier.empty():
        current = frontier.get()
        #Check if the goal has been expanded
        if goal == current:
            print("Solution Found")
            break
        
        for child in graph.node_neighbours(current):
            child_cost = cost_so_far[current] + graph.distance_measure(current, child)
            if child not in came_from or child_cost < cost_so_far[child]:
                priority = child_cost
                frontier.put(child, priority)
                came_from[child] = current
                cost_so_far[child] = child_cost
        if frontier.empty():
            print("No Solution")
                
    return came_from

def a_star_search(graph, start, goal):
    frontier = PriorityQueue()
    frontier.put(start, 0)
    came_from = {}
    cost_so_far = {}
    
    came_from[start] = None
    cost_so_far[start] = 0
    
    while not frontier.empty():
        current = frontier.get()
        #Check if the goal has been expanded
        if goal == current:
            print("Solution Found")
            break
        
        for child in graph.node_neighbours(current):
            child_cost = cost_so_far[current] + graph.distance_measure(current, child)
            if child not in came_from or child_cost < cost_so_far[child]:
                priority = child_cost
                frontier.put(child, priority)
                came_from[child] = current
                cost_so_far[child] = child_cost + graph.distance_measure(goal, child)
        if frontier.empty():
            print("No Solution")
                
    return came_from

def path_construction(start, node, search_results):
    if node == start:
        return [node]
    else:
        list_ = [node]
        list_.extend(path_construction(start, search_results[node], search_results))
        return list_

In [151]:
#Define Problem
start =  80005
goal = 260365663
search_results = breadth_first_search(graph, start, goal)
path_1 = path_construction(start, goal, search_results)
#search_results = uniform_cost_search(graph, start, goal)
#path_2 = path_construction(start, goal, search_results)
#search_results = a_star_search(graph, start, goal)
#path_2 = path_construction(start, goal, search_results)

Solution Found


In [153]:
import folium
m = folium.Map(location=[ 59.9139, 10.7522], zoom_start=14)
folium.GeoJson(graph.node_path_geojson(path_1),
    style_function=lambda x: {
        'color' : 'black',
        'weight' : 3,
        'opacity': 1,
        'fillColor' : 'black'
        }).add_to(m)
m

In [77]:
import psycopg2
from psycopg2.extras import RealDictCursor
import json
conn_string = "host='localhost' dbname='osm' user='dcromp'"
# print the connection string we will use to connect
print ("Connecting to database\n->%s" % (conn_string))
 
# get a connection, if a connect cannot be made an exception will be raised here
conn = psycopg2.connect(conn_string)
 
# conn.cursor will return a cursor object, you can use this cursor to perform queries
cursor = conn.cursor()
print ("Connected!\n")

Connecting to database
->host='localhost' dbname='osm' user='dcromp'
Connected!



In [125]:
# execute our Query
cursor.execute("SELECT ST_AsText(ST_Transform(geom,4326)) FROM nodes WHERE id = 443702116")

In [126]:
records = cursor.fetchall()

In [144]:
records[0][0].replace('POINT(', ' ').replace(')', '').split(' ')[1:3]

['10.8501136', '59.9435552']

In [137]:
getattr(Point, 'POINT(10.8501136 59.9435552)')

AttributeError: type object 'Point' has no attribute 'POINT(10.8501136 59.9435552)'

In [37]:
test = tagger(records[0][5])
print(test)

{'highway': 'crossing', 'source': 'bing'}


In [36]:
def tagger(self, string):
        """
        Converts and osmosis tag into a python dictionary

        Args:
            Osmosis tag, Example: '"source"=>"bing", "highway"=>"crossing"'
        Returns:
            Python dictionary
            {'source':'bing', 'highway':'crossing'}
        """
        tag_list = string.replace('"','' ).replace(' ','' ).split(',')
        tuples = (x.split('=>') for x in tag_list) # Each tag value pair is now a list of tuples
        return{tuple_[0]:tuple_[1] for tuple_ in tuples}

In [152]:
graph.node_neighbours(80005)

[1611182156, 472276884, 80007]

In [1]:
from osmapi import OsmApi
from geojson import LineString, GeometryCollection, Point
import folium
from collections import deque
import bz2

In [ ]:
MyApi = OsmApi()

def Map(	self, min_lon, min_lat, max_lon, max_lat)
Download data in bounding box. Returns list of dict { type: node|way|relation, data: {} }.

In [ ]:
highway_tags = ['motorway', 'trunk', 'primary', 'secondary', 'tertiary', 'unclassified', 'residential',
                            'service', 'motorway_link', 'trunk_link', 'primary_link', 'secondary_link', 'tertiary_link',
                            'living_street', 'pedestrian', 'road']
#Raw Data
map_data = MyApi.Map(22.216864,60.416923,22.286005,60.451442)
way_data = [row for row in map_data if row['type'] == 'way']
node_data = [row for row in map_data if row['type'] == 'node']
highway_raw = [row for row in way_data if 'highway' in row['data']['tag'].keys()]
highway_data = [row for row in highway_raw for tag in highway_tags if tag in row['data']['tag']['highway']]
link_data = [row for row in way_data if 'highway' in row['data']['tag'].keys() and 'link' in row['data']['tag']['highway']]

In [ ]:
#Helper functions
def node2loc(node_id):
    for node in node_data:
        if node_id == node['data']['id']:
            return (node['data']['lon'], node['data']['lat'])

In [ ]:
#All Roads to GeoJson
roads = []
for highway in highway_data:
    road = []
    nodes = highway['data']['nd'] #List of nodes in highway
    for node in nodes:
        road.append(node2loc(node))
    roads.append(LineString(road))
geo_json = GeometryCollection(roads)

In [ ]:
m = folium.Map(location=[ 60.4518, 22.2666], zoom_start=15)
folium.GeoJson(geo_json,
    style_function=lambda x: {
        'color' : 'black',
        'weight' : 3
        ,
        'opacity': 1,
        'fillColor' : 'black',
        }).add_to(m)

In [ ]:
m

In [ ]:
#Road turn
turns = set()
for highway_1 in highway_data:
    nodes_1 = highway_1['data']['nd']
    for node_1 in nodes_1:
        for highway_2 in highway_data: 
            if highway_2 != highway_1:
                nodes_2 = highway_2['data']['nd']
                for node_2 in nodes_2:
                    if node_2 == node_1:
                        turns.add(node2loc(node_1))

turn_geojson = GeometryCollection([Point(turn) for turn in list(turns)])

In [ ]:
m = folium.Map(location=[60.4518, 22.2666], zoom_start=15)
folium.GeoJson(geo_json,
    style_function=lambda x: {
        'color' : 'black',
        'weight' : 3
        ,
        'opacity': 1,
        'fillColor' : 'black',
        }).add_to(m)
folium.GeoJson(turn_geojson,
    style_function=lambda x: {
        'color' : 'black',
        'weight' : 3
        ,
        'opacity': 1,
        'fillColor' : 'black',
        }).add_to(m)

In [ ]:
m

In [66]:
from osmapi import OsmApi
from geojson import LineString, GeometryCollection, Point
from math import radians, sin, cos, sqrt, asin
import heapq
import numpy as np

class GraphPlan(object):
    """ Process data in a bounding box to produce data structures needed for
        a graph search
    """
    def __init__(self, lon1,  lat1, lon2, lat2):
        self.MyApi = OsmApi()
        
        #List of allowed road types
        self.highway_tags = ['motorway', 'trunk', 'primary', 'secondary', 'tertiary', 'unclassified', 'residential',
                            'service', 'motorway_link', 'trunk_link', 'primary_link', 'secondary_link', 'tertiary_link',
                            'living_street', 'pedestrian', 'road']
        
        #Extract basic data units
        self.bounding_box = [lon1,  lat1, lon2, lat2]
        self.map_data = self.MyApi.Map(lon1,  lat1, lon2, lat2)
        self.way_data = [row for row in self.map_data if row['type'] == 'way']
        self.node_data = [row for row in self.map_data if row['type'] == 'node']
        self.highway_data = [row for row in self.way_data if 'highway' in row['data']['tag'].keys()]
        #self.highway_data = [row for row in self.highway_raw for tag in self.highway_tags if tag in row['data']['tag']['highway']]
        
        #Node_id to lon lat
        self.node_ids = [node['data']['id'] for node in self.node_data]
        #self.node_id_loc = {node_id:self.node2loc(node_id) for node_id in self.node_ids}
        
        #Dictionary of road_id and list of node ID's
        self.roads = {highway['data']['id']:highway['data']['nd'] for highway in self.highway_data}
        
        #Dictionary of node_id and list of road_ID's
        self.node_roads = self.__node_roads_dict()
        
        #List of unique nodes at road changes or turns
        self.turns = self.__turn_extractor(self.roads)
        
        #Road_id and their turns
        self.road_turns = self.__road_turns(self.roads)
        
        self.neighbours = self.__neigbour_nodes()
        
        
    #Private Methods
    def __road_turns(self, roads):
        road_turns = {}
        for key in self.roads.keys():
            temp_turns = []
            for node in roads[key]:
                if node in self.turns:
                    temp_turns.append(node)
            road_turns[key] = temp_turns
        return road_turns
    
    def __turn_extractor(self, roads):
        turns = set()
        for road_1 in self.roads.keys():
            nodes_1 = self.roads[road_1]
            for node_1 in nodes_1:
                for road_2 in self.roads.keys(): 
                    if road_2 != road_1:
                        nodes_2 = self.roads[road_2]
                        for node_2 in nodes_2:
                            if node_2 == node_1:
                                turns.add(node_1)
        return list(turns)
    
    def __neigbour_nodes(self):
        neighbours_dict = {}
        for road in self.roads:
            for index, node in enumerate(self.roads[road]):
                forward_node = index + 1
                backward_node = index - 1
                try:
                    forward = self.roads[road][forward_node]
                    if node not in neighbours_dict:
                        neighbours_dict[node] = [forward]
                    else:
                        list_ = neighbours_dict[node]
                        list_.append(forward)
                        neighbours_dict[node] = list_
                except:
                    pass
                if backward_node >= 0:
                    try:
                        backward = self.roads[road][backward_node]
                        if node not in neighbours_dict:
                            neighbours_dict[node] = [backward]
                        else:
                            list_ = neighbours_dict[node]
                            list_.append(backward)
                            neighbours_dict[node] = list_
                    except:
                        pass 
        return neighbours_dict
    
    def __node_roads_dict(self):
        node_roads = {}
        for road in list(self.roads.keys()):
            for node in self.roads[road]:
                if node not in list(node_roads.keys()):
                    node_roads[node] = [road]
                else:
                    list_ = node_roads[node]
                    list_.append(road)
                    node_roads[node] = list_
        return node_roads
    
    def __haversine(self, lat1, lon1, lat2, lon2):
        R = 6372.8 # Earth radius in kilometers
        dLat = radians(lat2 - lat1)
        dLon = radians(lon2 - lon1)
        lat1 = radians(lat1)
        lat2 = radians(lat2)
        a = sin(dLat/2)**2 + cos(lat1)*cos(lat2)*sin(dLon/2)**2
        c = 2*asin(sqrt(a))
        return R * c
    
    #Helper Methods
    def node2loc(self, node_id):
        for node in self.node_data:
            if node_id == node['data']['id']:
                return (node['data']['lon'], node['data']['lat'])
    
    def node2road(self, node_id):
        return self.node_roads[node_id] #List of roads node is in
    
    def neighbour_list(self, node_id):
        return self.neighbours[node_id]
    
    def node_path_geojson(self, node_list):
        roads = []
        for node in node_list:
            roads.append(self.node2loc(node))
        return LineString(roads)
    
    def distance_measure(self, node1, node2):
        lon1, lat1 = self.node2loc(node1)
        lon2, lat2 = self.node2loc(node2)
        lon = lon1 - lon2
        lat = lat1 - lat2
        return np.sqrt(np.square(lon) + np.square(lat))
    
    def node_distance(self, node1, node2):
        lon1, lat1 = self.node2loc(node1)
        lon2, lat2 = self.node2loc(node2)
        distance = self.__haversine(lat1, lon1, lat2, lon2)
        return distance
    
class PriorityQueue:
    def __init__(self):
        self.elements = []
    
    def empty(self):
        return len(self.elements) == 0
    
    def put(self, item, priority):
        heapq.heappush(self.elements, (priority, item))
    
    def get(self):
        return heapq.heappop(self.elements)[1]
    
class BusGraph(GraphPlan):
    """
    Turns bus routes into nodes for the graph search
    """
    def __init__(self, lon1,  lat1, lon2, lat2):
        GraphPlan.__init__(self, lon1,  lat1, lon2, lat2 )
        
        #Extract all raw bus routes
        self.route_data = [row for row in self.map_data if row['type'] == 'route']
        #self.route_data = [row for row in self.way_data if 'route' in row['data']['tag'].keys()]
        #self.bus_data = [row for row in self.route_data if 'bus' in row['data']['tag']['route']]

In [67]:
#Load data
#graph = GraphPlan(22.216864,60.416923,22.286005,60.451442)

graph = BusGraph(0.738144,51.805537,0.754623,51.822623)

In [69]:
graph.map_data

[{'data': {'changeset': 23408013,
   'id': 21095660,
   'lat': 51.8434493,
   'lon': 0.7073007,
   'tag': {},
   'timestamp': datetime.datetime(2014, 7, 1, 10, 18, 13),
   'uid': 360562,
   'user': 'haytigran',
   'version': 13,
   'visible': True},
  'type': 'node'},
 {'data': {'changeset': 18438078,
   'id': 260359417,
   'lat': 51.8072377,
   'lon': 0.7545462,
   'tag': {},
   'timestamp': datetime.datetime(2013, 10, 19, 17, 50, 18),
   'uid': 84263,
   'user': 'Robert Whittaker',
   'version': 6,
   'visible': True},
  'type': 'node'},
 {'data': {'changeset': 18438078,
   'id': 260359654,
   'lat': 51.8088626,
   'lon': 0.752404,
   'tag': {},
   'timestamp': datetime.datetime(2013, 10, 19, 17, 50, 18),
   'uid': 84263,
   'user': 'Robert Whittaker',
   'version': 6,
   'visible': True},
  'type': 'node'},
 {'data': {'changeset': 334624,
   'id': 260359655,
   'lat': 51.8124123,
   'lon': 0.7490726,
   'tag': {},
   'timestamp': datetime.datetime(2009, 2, 10, 19, 33, 57),
   'uid':

In [30]:
#Define Problem
start =  249745223
goal = 4756728248

In [31]:
#Define Problem
start =  249745223
goal = 4756728248
search_results = breadth_first_search(graph, start, goal)
path_1 = path_construction(start, goal, search_results)
#search_results = uniform_cost_search(graph, start, goal)
#path_2 = path_construction(start, goal, search_results)
#search_results = a_star_search(graph, start, goal)
#path_2 = path_construction(start, goal, search_results)

Solution Found


In [ ]:
graph.neighbour_list(1101337726)

In [ ]:
graph.neighbour_list(21646150)

In [ ]:
graph.roads[42684571]

In [ ]:
graph.node2road(21646230)

In [ ]:
graph.roads[22649715]

In [ ]:
path_1

In [ ]:
graph.node_path_geojson([21855869,
 554190049,
 1252511515,
 30070345,
 1252511439,
 1274616355,
 1252511546,
 1252511468,
 21402480,
 256371984,
 597376717,
 1334702233,
 21402481,
 984624826,
 1334702231,
 283313787,
 558546949,
 215546743,
 215546699,
 1334702241,
 471079552,
 21855854,
 530866234,
 756885115,
 1252511533,
 21855855,
 1101337653,
 21646201,
 3223179595,
 2601642846,
 3223179632,
 3223179631,
 597376713,
 1101337825,
 1099808365,
 1099808361,
 1101337543,
 1101337632,
 1252511543,
 21646150])

In [24]:
graph.roads

{23130112: [249745223,
  249745292,
  1027109907,
  249745293,
  249745294,
  249745295,
  249745296],
 4694017: [26999436, 1211068989],
 109814443: [1256105899,
  1256105582,
  1256105980,
  3220142547,
  1256105373,
  1156353183,
  1256105800,
  1256105870],
 158203908: [1704365536, 1344370079],
 158203909: [296459091, 1704365534, 1038247183],
 158203904: [1344370079, 1344370055, 1344370099, 1344370047, 1344370105],
 23130119: [249745346,
  249745378,
  249745379,
  1277275401,
  249745380,
  1277275403,
  249745381,
  1277275430,
  249745382,
  1277275435,
  249745383,
  1027107001,
  1027107004,
  1027107028,
  3223111084,
  1027107021,
  1027107038,
  1027106982,
  1027107025,
  1027106849,
  1027106859],
 23130122: [249745379, 249745396, 1344370061, 249745397, 162312189],
 4231179: [25728453, 773674206, 2294505469, 2294505506, 3664910229],
 4231180: [25248704,
  2294507651,
  773674205,
  25248702,
  651391048,
  3664909821,
  1278544858,
  30969911,
  3231910344,
  3664909823,
 

In [ ]:
graph.node_path_geojson([21646150, 1101337726, 243674925, 533774011, 32919954, 30390838, 21646152])

In [ ]:
neighbour_nodes = set() #Only collect unique neighbours
way_list = test.node2way(338507468)
for way_id in way_list:
    way = test.ways[way_id]
    way_index = way.index(338507468)
    forward_node = way_index + 1
    backward_node = way_index - 1
    try:
        forward = way[forward_node]
        neighbour_nodes.add(forward)
    except:
        pass
    try:
        if backward_node > 0:
            backward = way[backward_node]
            neighbour_nodes.add(backward)
    except:
        pass

In [ ]:
neighbour_nodes

In [ ]:
temp = [1,2,3]
temp[3]

In [ ]:
test.ways

In [ ]:
m

In [ ]:
deque([2,4]).sort()

In [ ]:
graph = Aplanner(0.738144,51.809888,0.750504,51.822623)

#Define Problem
start = 308546906
goal = 316303594

search_path = []
explored_nodes = set()


def a_star_search(graph, start, goal):
    frontier = PriorityQueue()
    frontier.put(start, 0)
    came_from = {}
    cost_so_far = {}
    
    came_from[start] = None
    cost_so_far[start] = 0
    
    while not frontier.empty():
            
        current = frontier.get()
        
        #Check if the goal has been expanded
        if goal == came_from[current]:
            print("Solution Found")
            break
        
        children = graph.neigbour_nodes(current)
        
        for child in children:
            
            child_cost = cost_so_far[current] + graph.node_distance(current, child) - graph.node_distance(child, goal)
            
            if child not in came_from or child_cost < cost_so_far[child]:
                
                priority = child_cost
                frontier.put(child, priority)
                came_from[child] = current
                cost_so_far[child] = child_cost

                
    return came_from

search_results = a_star_search(graph, start, goal)
path = path_construction(start, goal, search_results)

In [ ]:
for node in test.turns:
    print(test.neigbour_nodes(node))

In [ ]:
test.ways[25643156]

In [ ]:
node_ways = {}
node_list = []

for way in list(test.ways.keys()):
    for node in test.ways[way]:
        if node not in list(node_ways.keys()):
            node_ways[node] = [way]
            node_list.append(node)
        else:
            print("Node already in dict")
            list_ = node_ways[node]
            list_.append(way)
            node_ways[node] = list_
#node_ways

In [ ]:
test.ways[32702976]

In [ ]:
print(len(set(node_list)))
print(len(node_list))

In [ ]:
test.turns

In [ ]:
test.turns

In [ ]:
if 1583202304 in test.ways.keys():
    print('yes')

In [ ]:
for key in node_ways.keys():
    if len(node_ways[key]) > 1:
        print (len(node_ways[key]))

In [ ]:
test.node_ways[1583202304]

In [ ]:
def node2loc(node_id):
    for node in node_data:
        if node_id == node['data']['id']:
            return (node['data']['lon'], node['data']['lat'])

In [ ]:
node_data[0]

In [ ]:
for dict_ in map_nodes:
    if dict_['type'] == 'way':
        way = dict_
        if 'highway' in way['data']['tag'].keys():
            print(way, '\n')

In [ ]:
set_ = set()
for dict_ in map_nodes:
    set_.add(tuple(dict_['type']))
    